In [10]:
"""国内疫情数据解析"""
import json,pandas,time,requests
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=&_=%d'%int(time.time()*1000)
data_1 = json.loads(requests.get(url=url).json()['data'])
date = data_1["lastUpdateTime"][0:10]
china_province = data_1["areaTree"][0]["children"]
china_data = pandas.DataFrame(columns=("地区","新增确诊","现有确诊","累计确诊","累计死亡","死亡率","累计治愈","治愈率"))
for i in china_province:
    area_name = i["name"]
    today_confirm = i["today"]["confirm"]
    nowConfirm = i["total"]["nowConfirm"]
    total_confirm = i["total"]["confirm"]
    total_dead = i["total"]["dead"]
    deadRate = i["total"]["deadRate"]
    total_heal = i["total"]["heal"]
    healRate = i["total"]["healRate"]
    china_data = china_data.append(pandas.DataFrame({"地区":area_name,"新增确诊":today_confirm,"现有确诊":nowConfirm,
                                                     "累计确诊":total_confirm,"累计死亡":total_dead,"死亡率":deadRate,
                                                     "累计治愈":total_heal,"治愈率":healRate},index=[0]))
    for j in i["children"]:
        area_name = j["name"]
        today_confirm = j["today"]["confirm"]
        nowConfirm = j["total"]["nowConfirm"]
        total_confirm = j["total"]["confirm"]
        total_dead = j["total"]["dead"]
        deadRate = j["total"]["deadRate"]
        total_heal = j["total"]["heal"]
        healRate = j["total"]["healRate"]
        china_data = china_data.append(pandas.DataFrame({"地区":area_name,"新增确诊":today_confirm,"现有确诊":nowConfirm,
                                                     "累计确诊":total_confirm,"累计死亡":total_dead,"死亡率":deadRate,
                                                     "累计治愈":total_heal,"治愈率":healRate},index=[0]))
china_data.to_csv("./china_data_{}.csv".format(date), index=False)

In [11]:
"""海外疫情数据解析"""
import time
url = "https://api.inews.qq.com/newsqa/v1/automation/foreign/country/ranklist"
data_2 = json.loads(requests.post(url).text)["data"]
overseas_data = pandas.DataFrame(columns=("国家","所属洲","新增确诊","现有确诊","累计确诊","累计死亡","累计治愈","新增现有确诊",
                                         "新增治愈", "新增死亡"))
times = time.strftime("%Y-%m-%d", time.localtime())
for i in data_2:
    overseas_name = i["name"]
    overseas_continent = i["continent"]
    new_confirm = i["confirmAdd"]
    total_confirm = i["confirm"]
    total_dead = i["dead"]
    total_heal = i["heal"]
    now_confirm = i["nowConfirm"]
    new_now_confirm = i["nowConfirmCompare"]
    new_heal = i["healCompare"]
    new_head = i["deadCompare"]
    overseas_data = overseas_data.append(pandas.DataFrame({"国家":overseas_name,"所属洲":overseas_continent,"新增确诊":new_confirm,
                                                           "现有确诊":now_confirm,"累计确诊":total_confirm,"累计死亡":total_dead,
                                                           "累计治愈":total_heal,"新增现有确诊":new_now_confirm,
                                                           "新增治愈":new_heal, "新增死亡":new_head}, index=[0]))
overseas_data.to_csv("./overseas_data_{}.csv".format(times), index=False)

In [18]:
"""国内疫情数据分析"""
from pyecharts.charts import Pie
import pyecharts.options as opts
df_china=pandas.read_csv("./china_data_{}.csv".format(date),header=0,index_col=0)
num = data_1['areaTree'][0]['children']
china_area = []
for i in num:
    china_area.append(i["name"])
china_confirm = []
for i in china_area:
    china_confirm.append(df_china.loc[i]["累计确诊"])
pie = (
    Pie()
    .add(series_name="{}日全国累计确诊数".format(date),
        data_pair=[(i,j) for i, j in zip(china_area, china_confirm)])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(legend_opts=opts.LegendOpts(is_show=False))
)
pie.render_notebook()

In [13]:
"""海外疫情数据分析"""
from pyecharts.charts import Pie
df_overseas=pandas.read_csv("./overseas_data_{}.csv".format(date),header=0)
data_3 = df_overseas.values.tolist()
overseas_country = []
overseas_confirm = []
for i in data_3:
    overseas_country.append(i[0])
    overseas_confirm.append(i[3])
pie = (
    Pie()
    .add(series_name="{}日海外累计确诊数".format(date),
        data_pair=[(i,j) for i, j in zip(overseas_country, overseas_confirm)])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(legend_opts=opts.LegendOpts(is_show=False))
)
pie.render_notebook()

In [14]:
"""海外累计确诊top10"""
from pyecharts.charts import Bar
df_top = df_overseas.sort_values(by="累计确诊", ascending=False).head(10)
df_top = df_top.values.tolist()
country=[]
confirm=[]
for i in range(len(df_top)):
    country.append(df_top[i][0])
    confirm.append(df_top[i][4])
bar = (
    Bar()
    .add_xaxis(country)
    .add_yaxis("累计确诊", confirm)
    .set_global_opts(title_opts = opts.TitleOpts(title="海外累计确诊top10"))
)
bar.render_notebook()

In [19]:
"""国内累计确诊top10"""
china_ten = pandas.DataFrame({"地区":china_area, "累计确诊":china_confirm}).sort_values(by="累计确诊", ascending=False).head(10)
china_ten = china_ten.values.tolist()
china_area_top = []
confirm = []
for i in range(10):
    china_area_top.append(china_ten[i][0])
#  这里由于吉林省和吉林市在表中都称为吉林，如果直接运行则会发生歧义
# 这里手动修改csv中第二个吉林为吉林市后重新运行"""国内疫情数据分析"""的代码,然后运行该部分就不会报错
    confirm.append(china_ten[i][1])  
    bar = (
        Bar()
        .add_xaxis(china_area_top)
        .add_yaxis("累计确诊", confirm)
        .set_global_opts(title_opts = opts.TitleOpts(title="国内累计确诊top10"))
    )
bar.render_notebook()

In [20]:
"""国内以往数据解析"""
import time, json, requests
from datetime import datetime
def catch_daily():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=wuwei_ww_cn_day_counts&callback=&_=%d'%int(time.time()*1000)
    data = json.loads(requests.get(url=url).json()['data'])
    data.sort(key=lambda x:x['date'])
    
    date_list = list()        # 日期
    confirm_list = list()     # 确诊
    suspect_list = list()     # 疑似
    dead_list = list()        # 死亡
    heal_list = list()        # 治愈
    for item in data:
        month, day = item['date'].split('/')
        date_list.append(datetime.strptime('2020-%s-%s'%(month, day), '%Y-%m-%d'))
        confirm_list.append(int(item['confirm']))
        suspect_list.append(int(item['suspect']))
        dead_list.append(int(item['dead']))
        heal_list.append(int(item['heal']))
    return date_list, confirm_list, suspect_list, dead_list, heal_list
def load_csv():
    # 获取数据
    date_list, confirm_list, suspect_list, dead_list, heal_list = catch_daily()
    # 获取当前日期命名(2020-02-13-daily.csv)
    n = time.strftime("%Y-%m-%d") + "-daily.csv"
    fw = open(n, 'w', encoding='utf-8')
    fw.write('date,confirm,suspect,dead,heal\n')
    i = 0
    while i<len(date_list):
        date = str(date_list[i].strftime("%Y-%m-%d"))
        fw.write(date+','+str(confirm_list[i])+','+str(suspect_list[i])+','+str(dead_list[i])+','+str(heal_list[i])+'\n')
        i = i + 1
    else:
        print("数据全部导入完毕")
        fw.close()

# 主函数
if __name__ == '__main__':
    load_csv()


数据全部导入完毕


In [21]:
"""国内每日疫情可视化"""
import time, json, requests
from datetime import datetime
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode
n = time.strftime("%Y-%m-%d") + "-daily.csv"
data = pd.read_csv(n)
date_list = list(data['date'])
confirm_list = list(data['confirm'])
suspect_list = list(data['suspect'])
dead_list = list(data['dead'])
heal_list = list(data['heal'])
line = (
    Line()
    .add_xaxis(date_list)
    .add_yaxis('累计确诊', confirm_list)
    .add_yaxis('疑似病例', suspect_list, is_smooth=True) #平滑
    .add_yaxis('累计死亡', dead_list)
    .add_yaxis('累计治愈', heal_list)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    # 设置x轴标签旋转角度
    .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-50)), 
                     yaxis_opts=opts.AxisOpts(name='人数'), 
                     title_opts=opts.TitleOpts(title='国内疫情数据'))        
    )
line.render_notebook()

In [22]:
"""中国疫情地图可视化"""
from pyecharts.charts import Map  # 导入地图的绘制库
import pyecharts.options as opts  # 全局相关的操作
china_data_map = pandas.DataFrame({"地区":china_area, "累计确诊":china_confirm})
china_data_map = china_data_map.values.tolist()
china_area_map = []
china_confirm_map = []
for i in china_data_map:
    china_area_map.append(i[0])
    china_confirm_map.append(i[1])
data_4  = zip(china_area_map, china_confirm_map)
map = (
    Map()
    .add("中国疫情地图", data_4, "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国疫情地图"),
        visualmap_opts=opts.VisualMapOpts(is_show=True,split_number=6, is_piecewise=True,
                                         pos_top='center',
                                              pieces=[
                                                   {'min': 10000, 'color': '#7f1818'},  #不指定 max
                                                   {'min': 1000, 'max': 10000},
                                                   {'min': 500, 'max': 999},
                                                   {'min': 100, 'max': 499},
                                                   {'min': 10, 'max': 99},
                                                   {'min': 0, 'max': 5} ])
    )
    )
map.render_notebook()

In [23]:
import json
import requests
import jsonpath
from pyecharts.charts import Map,Geo
from pyecharts import options as opts
from pyecharts.globals import GeoType,RenderType

# 1.目标网址
url = 'https://api.inews.qq.com/newsqa/v1/automation/foreign/country/ranklist'
# 2.模拟浏览器实现访问url
resp = requests.post(url).text
data = json.loads(resp)
# 3.从网页源代码中提取数据
name = jsonpath.jsonpath(data, "$..name")
print(len(name))
confirm = jsonpath.jsonpath(data, "$..confirm")
print(len(confirm))
a = zip(name, confirm)
nameMap = {
    'Singapore Rep.': '新加坡',
    'Dominican Rep.': '多米尼加',
    'Palestine': '巴勒斯坦',
    'Bahamas': '巴哈马',
    'Timor-Leste': '东帝汶',
    'Afghanistan': '阿富汗',
    'Guinea-Bissau': '几内亚比绍',
    "Côte d'Ivoire": '科特迪瓦',
    'Siachen Glacier': '锡亚琴冰川',
    "Br. Indian Ocean Ter.": '英属印度洋领土',
    'Angola': '安哥拉',
    'Albania': '阿尔巴尼亚',
    'United Arab Emirates': '阿联酋',
    'Argentina': '阿根廷',
    'Armenia': '亚美尼亚',
    'French Southern and Antarctic Lands': '法属南半球和南极领地',
    'Australia': '澳大利亚',
    'Austria': '奥地利',
    'Azerbaijan': '阿塞拜疆',
    'Burundi': '布隆迪',
    'Belgium': '比利时',
    'Benin': '贝宁',
    'Burkina Faso': '布基纳法索',
    'Bangladesh': '孟加拉国',
    'Bulgaria': '保加利亚',
    'The Bahamas': '巴哈马',
    'Bosnia and Herz.': '波斯尼亚和黑塞哥维那',
    'Belarus': '白俄罗斯',
    'Belize': '伯利兹',
    'Bermuda': '百慕大',
    'Bolivia': '玻利维亚',
    'Brazil': '巴西',
    'Brunei': '文莱',
    'Bhutan': '不丹',
    'Botswana': '博茨瓦纳',
    'Central African Rep.': '中非共和国',
    'Canada': '加拿大',
    'Switzerland': '瑞士',
    'Chile': '智利',
    'China': '中国',
    'Ivory Coast': '象牙海岸',
    'Cameroon': '喀麦隆',
    'Dem. Rep. Congo': '刚果（金）',
    'Congo': '刚果（布）',
    'Colombia': '哥伦比亚',
    'Costa Rica': '哥斯达黎加',
    'Cuba': '古巴',
    'N. Cyprus': '北塞浦路斯',
    'Cyprus': '塞浦路斯',
    'Czech Rep.': '捷克',
    'Germany': '德国',
    'Djibouti': '吉布提',
    'Denmark': '丹麦',
    'Algeria': '阿尔及利亚',
    'Ecuador': '厄瓜多尔',
    'Egypt': '埃及',
    'Eritrea': '厄立特里亚',
    'Spain': '西班牙',
    'Estonia': '爱沙尼亚',
    'Ethiopia': '埃塞俄比亚',
    'Finland': '芬兰',
    'Fiji': '斐',
    'Falkland Islands': '福克兰群岛',
    'France': '法国',
    'Gabon': '加蓬',
    'United Kingdom': '英国',
    'Georgia': '格鲁吉亚',
    'Ghana': '加纳',
    'Guinea': '几内亚',
    'Gambia': '冈比亚',
    'Guinea Bissau': '几内亚比绍',
    'Eq. Guinea': '赤道几内亚',
    'Greece': '希腊',
    'Greenland': '格陵兰',
    'Guatemala': '危地马拉',
    'French Guiana': '法属圭亚那',
    'Guyana': '圭亚那',
    'Honduras': '洪都拉斯',
    'Croatia': '克罗地亚',
    'Haiti': '海地',
    'Hungary': '匈牙利',
    'Indonesia': '印度尼西亚',
    'India': '印度',
    'Ireland': '爱尔兰',
    'Iran': '伊朗',
    'Iraq': '伊拉克',
    'Iceland': '冰岛',
    'Israel': '以色列',
    'Italy': '意大利',
    'Jamaica': '牙买加',
    'Jordan': '约旦',
    'Japan': '日本',
    'Japan': '日本本土',
    'Kazakhstan': '哈萨克斯坦',
    'Kenya': '肯尼亚',
    'Kyrgyzstan': '吉尔吉斯斯坦',
    'Cambodia': '柬埔寨',
    'Korea': '韩国',
    'Kosovo': '科索沃',
    'Kuwait': '科威特',
    'Lao PDR': '老挝',
    'Lebanon': '黎巴嫩',
    'Liberia': '利比里亚',
    'Libya': '利比亚',
    'Sri Lanka': '斯里兰卡',
    'Lesotho': '莱索托',
    'Lithuania': '立陶宛',
    'Luxembourg': '卢森堡',
    'Latvia': '拉脱维亚',
    'Morocco': '摩洛哥',
    'Moldova': '摩尔多瓦',
    'Madagascar': '马达加斯加',
    'Mexico': '墨西哥',
    'Macedonia': '马其顿',
    'Mali': '马里',
    'Myanmar': '缅甸',
    'Montenegro': '黑山',
    'Mongolia': '蒙古',
    'Mozambique': '莫桑比克',
    'Mauritania': '毛里塔尼亚',
    'Malawi': '马拉维',
    'Malaysia': '马来西亚',
    'Namibia': '纳米比亚',
    'New Caledonia': '新喀里多尼亚',
    'Niger': '尼日尔',
    'Nigeria': '尼日利亚',
    'Nicaragua': '尼加拉瓜',
    'Netherlands': '荷兰',
    'Norway': '挪威',
    'Nepal': '尼泊尔',
    'New Zealand': '新西兰',
    'Oman': '阿曼',
    'Pakistan': '巴基斯坦',
    'Panama': '巴拿马',
    'Peru': '秘鲁',
    'Philippines': '菲律宾',
    'Papua New Guinea': '巴布亚新几内亚',
    'Poland': '波兰',
    'Puerto Rico': '波多黎各',
    'Dem. Rep. Korea': '朝鲜',
    'Portugal': '葡萄牙',
    'Paraguay': '巴拉圭',
    'Qatar': '卡塔尔',
    'Romania': '罗马尼亚',
    'Russia': '俄罗斯',
    'Rwanda': '卢旺达',
    'W. Sahara': '西撒哈拉',
    'Saudi Arabia': '沙特阿拉伯',
    'Sudan': '苏丹',
    'S. Sudan': '南苏丹',
    'Senegal': '塞内加尔',
    'Solomon Is.': '所罗门群岛',
    'Sierra Leone': '塞拉利昂',
    'El Salvador': '萨尔瓦多',
    'Somaliland': '索马里兰',
    'Somalia': '索马里',
    'Serbia': '塞尔维亚',
    'Suriname': '苏里南',
    'Slovakia': '斯洛伐克',
    'Slovenia': '斯洛文尼亚',
    'Sweden': '瑞典',
    'Swaziland': '斯威士兰',
    'Syria': '叙利亚',
    'Chad': '乍得',
    'Togo': '多哥',
    'Thailand': '泰国',
    'Tajikistan': '塔吉克斯坦',
    'Turkmenistan': '土库曼斯坦',
    'East Timor': '东帝汶',
    'Trinidad and Tobago': '特里尼达和多巴哥',
    'Tunisia': '突尼斯',
    'Turkey': '土耳其',
    'Tanzania': '坦桑尼亚',
    'Uganda': '乌干达',
    'Ukraine': '乌克兰',
    'Uruguay': '乌拉圭',
    'United States': '美国',
    'Uzbekistan': '乌兹别克斯坦',
    'Venezuela': '委内瑞拉',
    'Vietnam': '越南',
    'Vanuatu': '瓦努阿图',
    'West Bank': '西岸',
    'Yemen': '也门',
    'South Africa': '南非',
    'Zambia': '赞比亚',
    'Zimbabwe': '津巴布韦'
}
map_ = Map(opts.InitOpts(width='1200px', height='600px')).add(series_name="累计确诊",  # 设置提示框标签
                 data_pair=a,  # 输入数据
                 maptype="world",  # 设置地图类型为世界地图
                 name_map=nameMap,  # 添加映射
                 is_map_symbol_show=False  # 不显示标记点
                 )

# 设置系列配置项
map_.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示国家名称
# 设置全局配置项
map_.set_global_opts(title_opts=opts.TitleOpts(title="海外疫情情况"),  # 设置图标题
                    visualmap_opts=opts.VisualMapOpts(max_=1000000, is_piecewise=True))  # 显示图例

map_.set_global_opts(title_opts=opts.TitleOpts(title="国外疫情情况"),  # 设置图标题
                     visualmap_opts=opts.VisualMapOpts(pieces=[  # 自定义分组的分店和颜色
                             {"min": 900000, "color": "#800000"},
                             {"min": 50000, "max": 500000, "lable":'0.15~0.19', "color": "#AA0000"},  # 栗色
                             {"min": 10000, "max": 50000, "color": "#CC0000"},  # 耐火砖
                             {"min": 1000, "max": 10000, "color": "#FF0000"},  # 印度红
                             {"min": 0, "max": 1000, "color": "#FF3333"},  # 玫瑰棕色
                             {"max": 0, "color": "#FFCCCC"},  # 薄雾玫瑰
                     ],


                             is_piecewise=True))  # 显示分段式图例


map_.render_notebook()

182
182


In [24]:
"""美国历史数据解析"""
import requests,json
from pyecharts.charts import Line
import pyecharts.options as opts
url = "https://api.inews.qq.com/newsqa/v1/automation/foreign/daily/list?country=%E7%BE%8E%E5%9B%BD"
reponse = requests.post(url)
usa_data = json.loads(reponse.text)["data"]
usa_confirm = []
usa_date = []
usa_dead = []
usa_heal = []
usa_add = []
for i in usa_data:
    usa_date.append(i["date"])
    usa_confirm.append(i["confirm"])
    usa_dead.append(i["dead"])
    usa_heal.append(i["heal"])
    usa_add.append(i["confirm_add"])
line= (
    Line()
    .add_xaxis(usa_date)
    .add_yaxis("确诊人数", usa_confirm)
    .add_yaxis("死亡人数", usa_dead)
    .add_yaxis("治愈人数", usa_heal)
    .add_yaxis("新增人数", usa_add)
    .set_global_opts(title_opts=opts.TitleOpts(title='美国疫情数据可视化'))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
line.render_notebook()